In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

# Inference

> How to perform inference with a tabular example

We start by replicating the training from the [Tabular tutorial](https://docs.fast.ai/tutorial.tabular.html) most of details are skipped here so see that tutorial for more depth. We can then export the trained model for inference.

In [ ]:
from fastai.tabular.all import *

We can download a sample of this dataset with the usual `untar_data` command:

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#5) [Path('/Users/jdp/.fastai/data/adult_sample/adult.csv'),Path('/Users/jdp/.fastai/data/adult_sample/export.pkl'),Path('/Users/jdp/.fastai/data/adult_sample/models'),Path('/Users/jdp/.fastai/data/adult_sample/tabular_trained_v_0_1'),Path('/Users/jdp/.fastai/data/adult_sample/tabular_learner_trained_v_0_1')]

In [ ]:
df = pd.read_csv(path/'adult.csv')
df.shape

(32561, 15)

In [ ]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.360247,0.356319,0.835995,00:04


Now that we have a trained model we can export it to use for inference:

This saves a copy of the learner to the learners data path `learn.path`, using the python pickle function. Note that it's a good idea to keep a log of your model describing the steps you used to arrive at it. This could include a copy of the training notebook or a good description in a notebook.

In [ ]:
learn.export("tabular_learner_trained_v_0_1")
learn_path = learn.path
learn_path.ls()

(#5) [Path('/Users/jdp/.fastai/data/adult_sample/adult.csv'),Path('/Users/jdp/.fastai/data/adult_sample/export.pkl'),Path('/Users/jdp/.fastai/data/adult_sample/models'),Path('/Users/jdp/.fastai/data/adult_sample/tabular_trained_v_0_1'),Path('/Users/jdp/.fastai/data/adult_sample/tabular_learner_trained_v_0_1')]

----

## In production

> We copy/upload our trained learner to our production enviroment. 

Once there we can load it and start making predictions on unseen data.

Let's delete our learner object before loading to simulate the production enviroment.

> Warning: `load_learner` requires all your custom code be in the exact same place as when exporting your `Learner` (the main script, or the module you imported it from).

In [ ]:
del learn

learn = load_learner(learn_path/"tabular_learner_trained_v_0_1")

The learner loads with an empty data loader, so to get prediction we need to load data into a test data loader.

You can use the `test_dl` method of the `DataLoaders`. That dataframe does not need to have the dependent variable in its column.

In [ ]:
len(learn.dls.train_ds)

0

In production you would recieve data for inference from an external source for instance via an http request. 
You will want to check this data before putting it through your model.
Checking the model could include:

- Missing data.
- Unknown columns.
- Data that lies far outside the scope of your training data.

Here we use a copy of the dataframe from training as an example.

In [ ]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

Then `Learner.get_preds` will give you the predictions:

In [ ]:
learn.get_preds(dl=dl)

(tensor([[0.6570, 0.3430],
         [0.4480, 0.5520],
         [0.7964, 0.2036],
         ...,
         [0.6095, 0.3905],
         [0.8448, 0.1552],
         [0.8727, 0.1273]]),
 None)